In [1]:
! pip install faiss-cpu==1.7.4 mistralai==0.0.12

In [2]:
import methods 
import formats
import json
import os 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

google_calendar = methods.get_google_calendar_service()
calendar = methods.extractCalendar(google_calendar)
client = methods.set_up_MISTRAL(calendar)

Based on the calendar you provided, there is only one event scheduled for today, March 25, 2024. Here are the details for the event:

Event Name: Meeting
Start Time: 2024-03-25T11:00:00-04:00 (11:00 AM Eastern Time)
End Time: 2024-03-25T12:00:00-04:00 (12:00 PM Eastern Time)
Event ID: 'vu7thvdp67tg5ukjoubia7ivl8'

So, you have a meeting scheduled today from 11:00 AM to 12:00 PM Eastern Time.

In [3]:
query = input()

prompt2 = f"Extract intent from the following query meant for a virtual calendar assistant - {query}. Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."

messages = [ChatMessage(role="user", content=prompt2)]
stream_response = client.chat_stream(model="open-mistral-7b", messages=messages)

intent = ''
for chunk in stream_response:
    intent += chunk.choices[0].delta.content or ""
print(intent)

can you creat a meeting at 9am tomorrow
createEvent


In [4]:
if intent == 'createEvent':
    prompt_for_creation = f"{query} based on {calendar}"
    messages = [ChatMessage(role="user", content=prompt_for_creation)]
    
    stream_response = client.chat_stream(
        model="open-mistral-7b",
        messages=messages
    )
    
    response_content = ''
    for chunk in stream_response:
        response_content += chunk.choices[0].delta.content or ""
    
    json_response = json.loads(response_content)
    print(json_response)
    entities = dict(json_response)
    methods.createEvent(entities)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)